# 🇫🇷 Fine-tuning GPT-2 Français avec Nanochat

Ce notebook vous permet de fine-tuner le modèle GPT-2 français (`asi/gpt-fr-cased-base`) sur vos dialogues préparés via **Pipeline Manager**.

## 📋 Prérequis

Avant de lancer ce notebook, assurez-vous d'avoir :
1. ✅ Préparé votre dataset via Pipeline Manager (local)
2. ✅ Exporté le fichier `combined_dataset.jsonl`
3. ✅ Le fichier contient des dialogues au format JSON avec `messages`

**⚠️ IMPORTANT : Activez le GPU !**
1. Menu `Runtime` → `Change runtime type`
2. Sélectionnez `T4 GPU`
3. Cliquez `Save`

**Temps estimé :** ~10 minutes pour 300 iterations

## 1️⃣ Installation des dépendances

Installation de l'environnement complet (2-3 minutes)

In [1]:
# Vérifier le GPU
!nvidia-smi

# Installer Rust (requis pour le tokenizer)
print("📦 Installation de Rust...")
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
import os
os.environ['PATH'] = f"{os.environ['HOME']}/.cargo/bin:{os.environ['PATH']}"

# Cloner le repo nanochat
print("\n📥 Clonage du repo nanochat...")
!git clone https://github.com/karpathy/nanochat
%cd nanochat

# Checkout de la branche French
!git checkout feat/french-experiment

# Installer les dépendances Python
print("\n📦 Installation des dépendances Python...")
!pip install -q torch transformers tiktoken maturin

# Compiler le tokenizer Rust
print("\n🔨 Compilation du tokenizer Rust...")
!maturin develop --release --manifest-path rustbpe/Cargo.toml

print("\n✅ Installation terminée !")

Fri Oct 17 08:33:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 2️⃣ Upload de votre dataset

Uploadez votre fichier `combined_dataset.jsonl` généré par Pipeline Manager

In [ ]:
from google.colab import files
import shutil
from pathlib import Path

# Créer la structure de dossiers
data_dir = Path("data/Stage.FINAL")
data_dir.mkdir(parents=True, exist_ok=True)

# Upload du fichier
print("📁 Sélectionnez votre fichier combined_dataset.jsonl...")
uploaded = files.upload()

# Déplacer le fichier uploadé
for filename in uploaded.keys():
    target_path = data_dir / "combined_dataset.jsonl"
    shutil.move(filename, target_path)
    print(f"✅ Fichier {filename} copié dans {target_path}")

# Vérifier le contenu
import json
dialogues = []
with open(target_path, 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():
            dialogues.append(json.loads(line))

print(f"\n📊 Statistiques du dataset :")
print(f"  - Total dialogues: {len(dialogues)}")
print(f"  - Split train (90%): ~{int(len(dialogues) * 0.9)}")
print(f"  - Split val (10%): ~{int(len(dialogues) * 0.1)}")

# Afficher un exemple
if dialogues:
    print(f"\n📝 Exemple de dialogue :")
    example = dialogues[0]
    for msg in example.get('messages', [])[:4]:  # Premiers 4 messages
        role = msg['role'].upper()
        content = msg['content'][:100]  # Premiers 100 chars
        print(f"  {role}: {content}...")

## 3️⃣ Téléchargement du modèle GPT-2 Français

Téléchargement du modèle pré-entraîné depuis HuggingFace

In [ ]:
print("📥 Téléchargement du modèle GPT-2 Français (asi/gpt-fr-cased-base)...")
print("⏱️  Cela peut prendre 2-3 minutes...\n")

!python -m scripts.download_gpt2_french

print("\n✅ Modèle téléchargé !")

## 4️⃣ Configuration du training

Paramètres par défaut (modifiables)

In [ ]:
# Paramètres de training
NUM_EPOCHS = 3                # Nombre d'époques (3 = bon compromis)
TARGET_EXAMPLES_PER_STEP = 8  # Nombre d'exemples par step (8 pour GPU)
DEVICE_BATCH_SIZE = 32        # Batch size (32 devrait passer sur T4 16GB)
EMBEDDING_LR = 0.2            # Learning rate pour embeddings
MATRIX_LR = 0.02              # Learning rate pour matrices
UNEMBEDDING_LR = 0.004        # Learning rate pour lm_head
EVAL_EVERY = 10               # Évaluation tous les N steps

# Calculer le nombre d'itérations approximatif
num_dialogues = len(dialogues)
train_dialogues = int(num_dialogues * 0.9)
approx_iterations = (train_dialogues // TARGET_EXAMPLES_PER_STEP) * NUM_EPOCHS

print("⚙️ Configuration :")
print(f"  - Epochs: {NUM_EPOCHS}")
print(f"  - Examples per step: {TARGET_EXAMPLES_PER_STEP}")
print(f"  - Device batch size: {DEVICE_BATCH_SIZE}")
print(f"  - Training dialogues: ~{train_dialogues}")
print(f"  - Iterations approximatives: ~{approx_iterations}")
print(f"  - Temps estimé (T4): ~{approx_iterations * 15 / 60:.1f} minutes")

## 5️⃣ Lancement du training

Le training va commencer. Vous verrez les logs en temps réel.

**Ce que vous allez voir :**
- Chargement du modèle GPT-2 français
- Chargement du tokenizer
- Chargement du dataset
- Progress avec validation loss tous les 10 steps
- Sauvegarde automatique du checkpoint final

In [ ]:
import time
start_time = time.time()

# Lancer le training
!python -m scripts.sft_gpt2_french \
  --num_epochs={NUM_EPOCHS} \
  --target_examples_per_step={TARGET_EXAMPLES_PER_STEP} \
  --device_batch_size={DEVICE_BATCH_SIZE} \
  --embedding_lr={EMBEDDING_LR} \
  --matrix_lr={MATRIX_LR} \
  --unembedding_lr={UNEMBEDDING_LR} \
  --eval_every={EVAL_EVERY} \
  --dtype=bfloat16

elapsed = time.time() - start_time
print(f"\n✅ Training terminé en {elapsed/60:.1f} minutes !")

## 6️⃣ Test du modèle fine-tuné

Testons le modèle avec quelques prompts

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import glob
from pathlib import Path

# Trouver le dernier checkpoint
print("📥 Chargement du modèle fine-tuné...")

checkpoint_dir = Path.home() / ".cache/nanochat/sft_french_checkpoints"
model_checkpoints = sorted(checkpoint_dir.glob("*/model_*.pt"))

if model_checkpoints:
    latest_checkpoint = model_checkpoints[-1]
    print(f"✅ Checkpoint trouvé : {latest_checkpoint}")

    # Charger le modèle de base
    model = GPT2LMHeadModel.from_pretrained("asi/gpt-fr-cased-base")

    # Charger les poids fine-tunés
    checkpoint = torch.load(latest_checkpoint, map_location='cuda')
    model.load_state_dict(checkpoint['model'])
    model.cuda()
    model.eval()

    tokenizer = GPT2Tokenizer.from_pretrained("asi/gpt-fr-cased-base")

    # Fonction de génération
    def generate(prompt, max_length=150, temperature=0.8, top_p=0.9):
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=max_length,
                temperature=temperature,
                do_sample=True,
                top_p=top_p,
                pad_token_id=tokenizer.eos_token_id
            )
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Tests
    print("\n" + "="*60)
    print("🤖 Test du modèle fine-tuné")
    print("="*60)

    test_prompts = [
        "Bonjour, comment allez-vous ?",
        "Expliquez-moi le machine learning",
        "Quelle est la capitale de la France ?"
    ]

    for prompt in test_prompts:
        print(f"\n📝 Prompt: \"{prompt}\"")
        result = generate(prompt)
        print(f"🤖 Génération: {result}")
        print("-" * 60)
else:
    print("❌ Aucun checkpoint trouvé. Le training a-t-il réussi ?")

## 7️⃣ Mode interactif

Testez le modèle avec vos propres prompts

In [ ]:
# Mode interactif
print("💬 Mode interactif activé !")
print("Entrez vos prompts ci-dessous (laissez vide pour arrêter)\n")

while True:
    prompt = input("\n🧑 Votre prompt : ")
    if not prompt.strip():
        break

    result = generate(prompt, max_length=200)
    print(f"\n🤖 {result}")
    print("-" * 60)

## 8️⃣ Télécharger le checkpoint

Téléchargez le modèle fine-tuné sur votre machine locale

In [ ]:
from google.colab import files
from pathlib import Path
import os

# Trouver tous les checkpoints
checkpoint_dir = Path.home() / ".cache/nanochat/sft_french_checkpoints"
model_files = sorted(checkpoint_dir.glob("*/model_*.pt"))
meta_files = sorted(checkpoint_dir.glob("*/meta_*.json"))

print(f"📦 Fichiers trouvés :")
print(f"  - {len(model_files)} model checkpoints")
print(f"  - {len(meta_files)} metadata files")

if model_files:
    # Télécharger le dernier checkpoint
    latest_model = model_files[-1]
    latest_meta = meta_files[-1] if meta_files else None

    print(f"\n⬇️ Téléchargement du dernier checkpoint...")
    files.download(str(latest_model))

    if latest_meta:
        files.download(str(latest_meta))

    print("\n✅ Checkpoint téléchargé !")
    print(f"📁 Fichier: {latest_model.name}")

    # Afficher la taille
    size_mb = latest_model.stat().st_size / (1024*1024)
    print(f"📊 Taille: {size_mb:.2f} MB")
else:
    print("❌ Aucun checkpoint trouvé")

## 📊 Statistiques du training

Résumé des performances

In [ ]:
import json
from pathlib import Path

# Charger le dernier meta.json
checkpoint_dir = Path.home() / ".cache/nanochat/sft_french_checkpoints"
meta_files = sorted(checkpoint_dir.glob("*/meta_*.json"))

if meta_files:
    with open(meta_files[-1], 'r') as f:
        meta = json.load(f)

    print("="*60)
    print("📊 RÉSUMÉ DU TRAINING")
    print("="*60)
    print(f"\n✅ Step final: {meta.get('step', 'N/A')}")
    print(f"✅ Loss final: {meta.get('val_loss', 'N/A'):.4f}")

    print(f"\n🎯 Modèle: GPT-2 French (asi/gpt-fr-cased-base)")
    print(f"🎯 Dataset: {len(dialogues)} dialogues")
    print(f"🎯 GPU: {torch.cuda.get_device_name(0)}")
    print(f"🎯 Epochs: {NUM_EPOCHS}")
    print("="*60)
else:
    print("❌ Aucun fichier meta.json trouvé")

## 🎯 Prochaines étapes

Votre modèle est maintenant fine-tuné ! Vous pouvez :

1. **Continuer le training** : Augmentez `NUM_EPOCHS` ou uploadez plus de dialogues
2. **Tester différents prompts** : Utilisez la cellule interactive
3. **Télécharger le checkpoint** : Pour l'utiliser localement
4. **Ajuster les hyperparamètres** : Modifiez learning rates, batch size, etc.

### Utiliser le checkpoint localement

```python
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Charger le modèle de base
model = GPT2LMHeadModel.from_pretrained("asi/gpt-fr-cased-base")

# Charger les poids fine-tunés
checkpoint = torch.load('model_000300.pt')
model.load_state_dict(checkpoint['model'])
model.eval()

# Générer du texte
tokenizer = GPT2Tokenizer.from_pretrained("asi/gpt-fr-cased-base")
# ... génération ...
```

### Workflow complet recommandé

```
LOCAL (Mac) :
├─ Pipeline Manager (http://localhost:8800)
│  ├─ Audio → Transcription → Édition
│  ├─ Génération dialogues (DeepSeek API)
│  ├─ Scoring automatique
│  └─ Export combined_dataset.jsonl
│
COLAB (GPU gratuit) :
├─ Upload combined_dataset.jsonl
├─ Fine-tuning (~10 min sur T4)
└─ Download checkpoint
```

### Ressources

- 📚 Pipeline Manager : `QUICKSTART_PIPELINE.md`
- 🏗️ Architecture : `ARCHITECTURE_GPT2_VS_NANOCHAT.md`
- 🔧 Troubleshooting : `TROUBLESHOOTING_FR.md`
- ☁️ Guide cloud GPU : `GUIDE_CLOUD_GPU.md`

---

**🎉 Félicitations ! Vous avez maintenant un modèle GPT-2 français personnalisé !**